<a href="https://colab.research.google.com/github/arjygh/stocksandoptions/blob/Working_AG/TA-options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#optional installations: 
!pip install yfinance --upgrade --no-cache-dir
# !pip3 install pandas_datareader
# !pip3 install matplotlib


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import ssl
from urllib import request
#dummy change


#newest yahoo API 
import yfinance as yahoo_finance

#optional 
yahoo_finance.pdr_override()

%matplotlib inline

     |████████████████████████████████| 5.5MB 9.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=be5f2bd9db4c9037b2fff5752f273d6abfb534d1ccd4adc5b42b86bad307db89
  Stored in directory: /tmp/pip-ephem-wheel-cache-i36c9jpd/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [12]:
def computeRSI (dataframe, time_window):
    diff = dataframe.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)

    return rsi


def stochastics( dataframe, low, high, close, k, d ):
    """
    Fast stochastic calculation
    %K = (Current Close - Lowest Low)/
    (Highest High - Lowest Low) * 100
    %D = 3-day SMA of %K

    Slow stochastic calculation
    %K = %D of fast stochastic
    %D = 3-day SMA of %K

    When %K crosses above %D, buy signal 
    When the %K crosses below %D, sell signal
    """

    df = dataframe.copy()

    # Set minimum low and maximum high of the k stoch
    low_min  = df[low].rolling( window = k ).min()
    high_max = df[high].rolling( window = k ).max()

    # Fast Stochastic
    df['k_fast'] = 100 * (df[close] - low_min)/(high_max - low_min)
    df['d_fast'] = df['k_fast'].rolling(window = d).mean()

    # Slow Stochastic
    df['k_slow'] = df["d_fast"]
    df['d_slow'] = df['k_slow'].rolling(window = d).mean()
    df['d_div'] = df['d_fast'] - df['d_slow']

    return df

def MACD(dataframe,close,span1,span2,sig_span):
    df = dataframe.copy()
    exp1 = df[close].ewm(span=span1, adjust=False).mean()
    exp2 = df[close].ewm(span=span2, adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=sig_span, adjust=False).mean()
    df['MACD line'] = macd
    df['Sig Line'] = exp3
    df['MACD_Div'] = macd-exp3
#     plt.plot(df.Date, macd, label='MACD', color = '#EBD2BE')
#     plt.plot(df.Date, exp3, label='Signal Line', color='#E5A4CB')
#     plt.legend(loc='upper left')
#     plt.show()
    return df

def CBI_func(ticker,start_time, end_time,trade_date_delta,slope1_delta,slope2_delta):
    # df = dataframe.copy()
    # day30 = df['Date'].iloc[-30]
    # start_time = datetime.datetime(2017, 10, 1)
    # #end_time = datetime.datetime(2019, 1, 20)
    # end_time = datetime.datetime.now().date().isoformat() 
    # start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
    # end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
    # print("Start Date: ", start_time)
    # print("End Date: ", end_time )
    # yahoo gives only daily historical data
    connected = False
    while not connected:
        try:
            # ticker_df = pdr.get_data_yahoo(ticker, start=start_time, end=end_time)
            ticker_df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
            # ticker_df = data.DataReader(ticker, 'google',start=start_time, end=end_time)
            connected = True
            # print('connected to yahoo')
        except Exception as e:
            print("Ticker: ", ticker,"  >>  Error: " + str(e))
            # time.sleep( 5 )
            continue 

    # use numerical integer index instead of date    
    ticker_df = ticker_df.reset_index()
    # print(ticker_df.head(5))
    df = ticker_df
    df['RSI'] = computeRSI(df['Adj Close'], 14)
    df = stochastics( df, 'Low', 'High', 'Close', 14, 3 )
    df = MACD(df,'Close',12,26,9)
    

    

    day0 = df['Date'].iloc[-1*trade_date_delta] #trade date
    day1 = df['Date'].iloc[-1*slope1_delta]
    day2 = df['Date'].iloc[-1*slope2_delta]
    print("day0 = ", day0)
    df = df[(df["Date"] < day0)]

    

    trade_date_price = df['Close'].iloc[-1*trade_date_delta]
    # print('df:',df)
    # max_price_30day = df.loc[1:30, 'Close'].max()
    holding_days = min(7,trade_date_delta)
    mask = df['Date'].between(day0,(day0.date() + datetime.timedelta(days = holding_days)).isoformat())
    max_price = df.loc[mask,'Close'].max()
    print("max_price: ", max_price)
    Price_delta = 100*(max_price - trade_date_price)/trade_date_price

    CBIL = [] #Call Buying Indicator List
    CBIL.append(day0)

    avg_volume = avg_volume_in_window(df, 15, day0)
    
    #trade date values
    RSI_val = df[df['Date']==day0]['RSI'].values
    MACD_Div_val = df[df['Date']==day0]['MACD_Div'].values
    d_slow_val = df[df['Date']==day0]['d_slow'].values

    
    #slopes
    RSI_fit = slope_calculator(df, 7, day0,'RSI')
    MACD_Div_fit = slope_calculator(df,3,day0,'MACD_Div')
    d_slow_fit = slope_calculator(df,3,day0,'d_slow')


    vol_limit = 2e6

    if avg_volume > vol_limit and RSI_val > 45 and RSI_val < 75:

      #RSI values: 
      AllRSI = np.array([df[df['Date']==day0]['RSI'].values,df[df['Date']==day1]['RSI'].values,df[df['Date']==day2]['RSI'].values])
      #d_slow values: 
      AllSS = np.array([df[df['Date']==day0]['d_slow'].values,df[df['Date']==day1]['d_slow'].values,df[df['Date']==day2]['d_slow'].values])
      #d_fast values
      AllSF = np.array([df[df['Date']==day0]['d_fast'].values,df[df['Date']==day1]['d_fast'].values,df[df['Date']==day2]['d_fast'].values])
      #d_div values
      AllSD = np.array([df[df['Date']==day0]['d_div'].values,df[df['Date']==day1]['d_div'].values,df[df['Date']==day2]['d_div'].values])
      #MACD_Div values
      AllMD = np.array([df[df['Date']==day0]['MACD_Div'].values,df[df['Date']==day1]['MACD_Div'].values,df[df['Date']==day2]['MACD_Div'].values])

      # row = df[df['Date']==end_time]['d_slow'].values
      # print(df['Date'].iloc[-1])
      # print(AllRSI, AllSS, AllSF, AllMD)

      #Algorithm for entry ot exit
    

      #RSI Score
      # if AllRSI[0] > 50 and AllRSI[0] < 75:
      #   CBIL.append((abs(AllRSI[0] - 50)/25).item())
      # else: 
      #   CBIL.append(1.0)
      
      ##SS score
      # SS7DayDiv = AllSS[0] - AllSS[1] #use only when there is an increasing trend
      # if AllSS[0] > 20 and AllSS[0]< 40 and SS7DayDiv > 0:
      #   CBIL.append((abs(AllSS[0]-30)/10).item())
      # else:
      #   CBIL.append(1.0)
      if d_slow_val > 20 and d_slow_val < 40 and d_slow_fit[0] > 0:
        CBIL.append((abs(AllSS[0]-30)/10).item())
      else: 
        CBIL.append(1.0)

      if MACD_Div_val < 0 and MACD_Div_fit[1] > 0 and MACD_Div_fit[0] < 0:
        CBIL.append(MACD_Div_val/MACD_Div_fit[1])
      else: 
        CBIL.append(1.0)

          

      # #MACD_Div
      # AllMDDiv = abs(AllMD[1]) - abs(AllMD[0])
      # if AllMD[0] < 0 and AllMD[1] < 0 and abs(AllMD[1]) > abs(AllMD[0]):
      #   CBIL.append((1-(abs(AllMD[1]) - abs(AllMD[0]))/abs(AllMD[1])).item())
      # else:
      #   CBIL.append(1.0)
      
      CBIL.append(trade_date_price)
      CBIL.append(Price_delta)
      CBI = (sum(CBIL[1:2])/2.0)
      CBIL.append(CBI)
      
      if CBI < 0.25:
          print("$$$$$ CBI = ",CBI,"$$$$$")
          print("_________________________________")
          print("RSI = ", AllRSI[0], AllRSI[1],AllRSI[2])
          print("Stochastic Slow =", AllSS[0], AllSS[1], AllSS[2])
          print("Stochastic Fast =", AllSF[0], AllSF[1], AllSF[2])
          print("MACD Div =", AllMD[0], AllMD[1], AllMD[2])
          print("Average Volume =", avg_volume)
          print("_________________________________")
    else:
      print("Volume lower than ",vol_limit)

            
    return CBIL

def avg_volume_in_window(dataframe, num_days, trade_date):
  df = dataframe.copy()
  df['Date'] = pd.to_datetime(df['Date'])

  start_time = (trade_date.date() - datetime.timedelta(days = num_days)).isoformat()
  df = df[(df["Date"] > start_time)]
  return df["Volume"].mean(axis=0)

def slope_calculator(dataframe, num_days, trade_date,indicator):
  df = dataframe.copy()
  df['Date'] = pd.to_datetime(df['Date'])
  start_time = (trade_date.date() - datetime.timedelta(days = num_days)).isoformat()
  df = df[(df["Date"] > start_time)]
  Y = np.array(df[indicator])
  X = np.array([n for n in range(0, len(Y))])
  Z = np.polyfit(X,Y,1)
  






In [14]:
start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
# end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
end_time = datetime.datetime.now().date().isoformat()
print("Start Date: ", start_time)
print("End Date: ", end_time )

ticker_list = ['AAPL']
#CBI_all = []
for ticker in ticker_list: 
    CBIL = CBI_func(ticker, start_time, end_time, 30, 3, 7)

print(CBIL, ";", sum(CBIL[1:2])/2.0)

Start Date:  2021-01-04
End Date:  2021-04-04
[*********************100%***********************]  1 of 1 completed
         Date        Open        High         Low       Close   Adj Close     Volume        RSI     k_fast     d_fast     k_slow     d_slow     d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  133.520004  133.610001  126.760002  129.410004  129.216919  143301900        NaN        NaN        NaN        NaN        NaN       NaN   0.000000  0.000000  0.000000
1  2021-01-05  128.889999  131.740005  128.429993  131.009995  130.814514   97664900        NaN        NaN        NaN        NaN        NaN       NaN   0.127635  0.025527  0.102108
2  2021-01-06  127.720001  131.050003  126.379997  126.599998  126.411102  155088000        NaN        NaN        NaN        NaN        NaN       NaN  -0.125616 -0.004702 -0.120914
3  2021-01-07  128.360001  131.630005  127.860001  130.919998  130.724655  109578200        NaN        NaN        NaN        NaN        NaN       NaN   0.022015 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
##START - only needed when  getting SSLVerificationError in local##
context = ssl._create_unverified_context()
response = request.urlopen(url, context=context)
html = response.read()
##END - only needed when  getting SSLVerificationError in local##
table=pd.read_html(html)
sp500 = table[0]
print (sp500)
sp500 = sp500[['Symbol','Security','GICS Sector']]
# sp500 = sp500.head(5)

# filename = 'constituents_csv.csv'
# sp500 = pd.read_csv(filename)
# sp500['CBI_RSI']= np.nan
sp500['CBI_SS']= np.nan
sp500['CBI_MACD']= np.nan
sp500['CBI'] = np.nan
sp500['t-30price'] = np.nan
sp500['Price_Delta'] = np.nan
sp500['Trade_Date'] = np.nan
sp500 = sp500[sp500['Symbol'] != "."]
# sp500 = sp500.head(100)

# sp500.groupby(['GICS Sector']).count()
# df2['CBI'] = CBI_func(df2['Symbol'],start_time,end_time)

# start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
# end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
print("Start Date: ", start_time)
print("End Date: ", end_time )

#
sector_list = list(sp500['GICS Sector'].unique())
#ticker_list = list(sp500['Symbol'])
passed_tickers=[]
no_data_tickers = []
for sector in sector_list:
    ticker_list = list(sp500[sp500['GICS Sector']==sector]['Symbol'])
    for ticker in ticker_list:
      try:
        CBIL = CBI_func(ticker, start_time, end_time,30,3,7)

        if len(CBIL) > 0:
          sp500.loc[sp500['Symbol'] == ticker, ['Trade_Date','CBI_SS','CBI_MACD','CBI','t-30price','Price_Delta']] = [CBIL[0], CBIL[1], CBIL[2], CBIL[5],CBIL[3],CBIL[4]]
        
      except:
        print('Analyzed Security: ',ticker, 'No Data')
        no_data_tickers.append(ticker)
        continue


#print("CBI = ", CBI_all)
print(sp500)
print('Tickers Qualified by CBI Criteria: ', passed_tickers)
print('Tickers for which data was not obtained: ', no_data_tickers)
sp500.plot(x='CBI',y='Price_Delta',style='o')

         Date   Open       High        Low      Close  Adj Close    Volume        RSI     k_fast     d_fast     k_slow     d_slow     d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  15.22  15.230000  14.805000  15.060000  15.060000   4414569        NaN        NaN        NaN        NaN        NaN       NaN   0.000000  0.000000  0.000000
1  2021-01-05  15.00  15.580000  14.970000  15.450000  15.450000   2106601        NaN        NaN        NaN        NaN        NaN       NaN   0.031111  0.006222  0.024889
2  2021-01-06  15.44  15.870000  15.380000  15.510000  15.510000   3744988        NaN        NaN        NaN        NaN        NaN       NaN   0.059918  0.016961  0.042956
3  2021-01-07  15.70  15.840000  15.420000  15.590000  15.590000   2340354        NaN        NaN        NaN        NaN        NaN       NaN   0.088186  0.031206  0.056980
4  2021-01-08  15.65  15.750000  15.250000  15.250000  15.250000   3293503        NaN        NaN        NaN        NaN        NaN       NaN   0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Volume lower than  2000000.0
Analyzed Security:  UA No Data
[*********************100%***********************]  1 of 1 completed
         Date       Open       High        Low      Close  Adj Close   Volume        RSI     k_fast     d_fast     k_slow     d_slow      d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  86.580002  87.129997  84.330002  85.120003  85.120003  2119400        NaN        NaN        NaN        NaN        NaN        NaN   0.000000  0.000000  0.000000
1  2021-01-05  84.050003  86.110001  84.050003  85.330002  85.330002  1503100        NaN        NaN        NaN        NaN        NaN        NaN   0.016752  0.003350  0.013402
2  2021-01-06  85.970001  88.239998  85.970001  87.480003  87.480003  2445700        NaN        NaN        NaN        NaN        NaN        NaN   0.201196  0.042920  0.158277
3  2021-01-07  88.419998  89.680000  88.000000  88.279999  88.279999  2063800        NaN        NaN        NaN        NaN        NaN        NaN   0.407228  0.115781  0.291

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


         Date        Open        High         Low       Close   Adj Close   Volume        RSI     k_fast     d_fast     k_slow     d_slow      d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  109.019997  109.019997  104.860001  105.820000  105.315231  2058200        NaN        NaN        NaN        NaN        NaN        NaN   0.000000  0.000000  0.000000
1  2021-01-05  105.639999  106.349998  105.199997  105.830002  105.325180  1475300        NaN        NaN        NaN        NaN        NaN        NaN   0.000798  0.000160  0.000638
2  2021-01-06  105.629997  106.699997  105.389999  106.410004  105.902420  1352700        NaN        NaN        NaN        NaN        NaN        NaN   0.047682  0.009664  0.038018
3  2021-01-07  106.500000  106.989998  105.279999  105.599998  105.096275  2222900        NaN        NaN        NaN        NaN        NaN        NaN   0.019255  0.011582  0.007673
4  2021-01-08  105.809998  107.449997  105.620003  107.150002  106.638885  1252200        NaN       

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


         Date       Open       High        Low      Close  Adj Close   Volume        RSI     k_fast     d_fast     k_slow     d_slow      d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  37.650002  37.950001  36.820000  37.130001  36.884716  7081000        NaN        NaN        NaN        NaN        NaN        NaN   0.000000  0.000000  0.000000
1  2021-01-05  37.279999  38.009998  37.180000  37.750000  37.500618  4268000        NaN        NaN        NaN        NaN        NaN        NaN   0.049459  0.009892  0.039567
2  2021-01-06  38.619999  40.660000  38.619999  40.209999  39.944366  8198600        NaN        NaN        NaN        NaN        NaN        NaN   0.283884  0.064690  0.219194
3  2021-01-07  40.779999  41.040001  40.349998  40.889999  40.619877  5109700        NaN        NaN        NaN        NaN        NaN        NaN   0.518560  0.155464  0.363096
4  2021-01-08  41.000000  41.119999  40.009998  40.619999  40.351658  5344800        NaN        NaN        NaN        NaN    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


         Date        Open        High         Low       Close   Adj Close  Volume        RSI     k_fast     d_fast     k_slow     d_slow     d_div  MACD line  Sig Line  MACD_Div
0  2021-01-04  195.300003  195.899994  185.800003  188.110001  187.200287  587800        NaN        NaN        NaN        NaN        NaN       NaN   0.000000  0.000000  0.000000
1  2021-01-05  188.110001  189.850006  185.710007  187.009995  186.105606  560900        NaN        NaN        NaN        NaN        NaN       NaN  -0.087750 -0.017550 -0.070200
2  2021-01-06  191.259995  199.940002  190.830002  197.740005  196.783722  736400        NaN        NaN        NaN        NaN        NaN       NaN   0.700456  0.126051  0.574405
3  2021-01-07  200.000000  202.179993  199.410004  200.130005  199.162155  585900        NaN        NaN        NaN        NaN        NaN       NaN   1.500668  0.400975  1.099694
4  2021-01-08  200.750000  201.639999  198.520004  201.550003  200.575287  545400        NaN        NaN       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
print("\n\n------- QUALIFIED BY RSI -----------")
print(sp500[sp500['CBI_RSI'] < 0.25])

print("\n\n------- QUALIFIED BY SS -----------")
print(sp500[sp500['CBI_SS'] < 0.25])

print("\n\n------- QUALIFIED BY MACD -----------")
print(sp500[sp500['CBI_MACD'] < 0.25])

print("\n\n------- QUALIFIED BY CBI -----------")
print(sp500[sp500['CBI'] < 0.25])




------- QUALIFIED BY RSI -----------
    Symbol                Security             GICS Sector   CBI_RSI  CBI_SS  CBI_MACD       CBI    t-30price  Price_Delta           Trade_Date
4      ACN               Accenture  Information Technology  0.232725     1.0       1.0  0.410908   257.679993     3.372401  2021-02-09 00:00:00
7      AMD  Advanced Micro Devices  Information Technology  0.059293     1.0       1.0  0.353098    90.910004     3.145961  2021-02-09 00:00:00
10     AFL                   Aflac              Financials  0.224997     1.0       1.0  0.408332    46.250000    10.291888  2021-02-09 00:00:00
25    AMZN         Amazon.com Inc.  Consumer Discretionary  0.181344     1.0       1.0  0.393781  3305.000000     0.702874  2021-02-09 00:00:00
39     ADI    Analog Devices, Inc.  Information Technology  0.096297     1.0       1.0  0.365432   152.850006     7.163884  2021-02-09 00:00:00
..     ...                     ...                     ...       ...     ...       ...       ... 